# **Data preparation**

## **Generating non redundant datasets**

In [55]:
!pip install biopython

In [56]:
import pandas as pd
import numpy as np
import os
from Bio import SeqIO

In [57]:
def filter_redundancy(unique_ids, total_dataset, cols):
    unique_ids = pd.read_csv(unique_ids, header=None, sep='\t', names=cols)
    total_df = pd.read_csv(total_dataset, header=None, sep='\t', names=cols)
    filtered = total_df[total_df.iloc[:,0].isin(unique_ids.iloc[:,0])]
    return filtered

positives_cols = ['id', 'organism_name', 'kingdom', 'sequence_length', 'cleavage_site']
negatives_cols = ['id', 'organism_name', 'kingdom', 'sequence_length', 'transmembrane']
non_redundant_pos = filter_redundancy('uniq.pos.tsv', 'positive.tsv', positives_cols)
non_redundant_neg = filter_redundancy('uniq.neg.tsv', 'negative.tsv', negatives_cols)
print(len(non_redundant_neg))
print(len(non_redundant_pos))

8934
1093


## **Dividing in train and test set**

In [58]:
def split_train_test(df: pd.DataFrame, split: int):
    random_seed = 42
    shuffle_df = df.sample(frac=1, random_state=random_seed)
    split_idx = split*len(df.index)//100
    training_set = shuffle_df.iloc[:split_idx,:].copy()
    test_set = shuffle_df.iloc[split_idx:,:].copy()
    return training_set, test_set

training_set_pos, test_set_pos = split_train_test(non_redundant_pos, 80)
training_set_neg, test_set_neg = split_train_test(non_redundant_neg, 80)

In [59]:
print(f'the lenght of the positive set is {len(training_set_pos)+len(test_set_pos)}; training:{len(training_set_pos)}, testing{len(test_set_pos)}')
print(f'the lenght of the negative set is {len(training_set_neg)+len(test_set_neg)}; training:{len(training_set_neg)}, testing{len(test_set_neg)}')


the lenght of the positive set is 1093; training:874, testing219
the lenght of the negative set is 8934; training:7147, testing1787


## **Dividing the training set in 5 validation sets**

In [60]:
bins = [0, 25, 50, 75, 100]
bin_lables = [1, 2, 3, 4, 5]
training_set_pos.loc[:, 'temp_row_num'] = training_set_pos.index.to_numpy(dtype=np.int64)

training_set_pos.loc[:, 'validation_n'] = pd.qcut(training_set_pos['temp_row_num'], q=5 , labels=bin_lables )

training_set_neg.loc[:, 'temp_row_num'] = training_set_neg.index.to_numpy(dtype=np.int64)

training_set_neg.loc[:, 'validation_n'] = pd.qcut(training_set_neg['temp_row_num'], q=5 , labels=bin_lables )

In [61]:
training_set_pos["sp_type"] = 1
training_set_neg["sp_type"] = 0
test_set_pos["sp_type"] = 1
test_set_neg["sp_type"] = 0

In [62]:
training_set_neg = training_set_neg.drop(columns=['temp_row_num'])
training_set_pos = training_set_pos.drop(columns=['temp_row_num'])

In [63]:
training_set = pd.concat([training_set_pos, training_set_neg])
test_set = pd.concat([test_set_pos, test_set_neg])

training_set.to_csv('training_set.tsv', sep='\t', index=False)
test_set.to_csv('test_set.tsv', sep='\t', index=False)